In [ ]:
import torch
from RealESRGAN import RealESRGAN
import os
import torch
import argparse
import itertools
import numpy as np
from tqdm import tqdm
import torch.optim as optim
from torchvision.utils import save_image
from torch import Tensor
from torchvision import transforms
from torch.utils.data import DataLoader,Dataset
from glob import glob
from torch.utils.data.distributed import DistributedSampler
import random
from PIL import Image
import torchvision
import torch.optim as optim
import torch.nn.functional as F
import torch.nn as nn
from torchvision import models
from matplotlib import pyplot as plt
import nibabel as nib
from cyclegan_3d.base_model import BaseModel
from cyclegan_3d.cycle_gan_model import CycleGANModel
from cyclegan_3d.networks3D import define_G, define_D
print(f"GPUs used:\t{torch.cuda.device_count()}")
device = torch.device("cuda",0)
print(f"Device:\t\t{device}")
import pytorch_model_summary as tms
import random
import cv2
def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)


<font size="6">lschemic</font>

In [ ]:
params = {
    # ✅ 데이터 설정
    'data_path': '../../data/registration_data/',     # Train images path
    'val_path': '../../data/registration_data',       # Validation images path
    'img_form': 'nii.gz',                                    # 이미지 포맷
    'batch_size': 1,
    'patch_size': [64,128, 128],                            # 3D 패치 크기 (H, W, D)
    'input_nc': 1,                                           # 입력 채널 수
    'output_nc': 1,                                          # 출력 채널 수
    'resample': False,
    'new_resolution': (0.45, 0.45, 0.45),
    'min_pixel': 0.1,
    'drop_ratio': 0,

    # ✅ 모델 구조 설정
    'ngf': 64,
    'ndf': 64,
    'netG': 'Dynet',
    'netD': 'n_layers',
    'n_layers_D': 3,
    'norm': 'instance',
    'no_dropout': True,

    # ✅ 학습 관련
    'isTrain': True,
    'model': 'cycle_gan',
    'direction': 'AtoB',
    'which_direction': 'AtoB',
    'phase': 'train',
    'gpu_ids': [0],
    'device': torch.device("cuda:0" if torch.cuda.is_available() else "cpu"),
    'workers': 4,

    'niter': 5000,                  # 학습 유지 epoch 수
    'niter_decay': 500,            # 학습률 감소 epoch 수
    'epoch_count': 1,
    'which_epoch': 'latest',
    'continue_train': False,
    'lr': 2e-4,
    'beta1': 0.5,
    'lr_policy': 'lambda',
    'lr_decay_iters': 50,
    'no_lsgan': False,
    'pool_size': 0,
    'lambda_A': 10.0,
    'lambda_B': 10.0,
    'lambda_identity': 0.,

    # ✅ 초기화
    'init_type': 'normal',
    'init_gain': 0.02,

    # ✅ 저장/출력
    'checkpoints_dir': '../../model/translation/',
    'name': '3D_MRI_CT',
    'print_freq': 100,
    'save_latest_freq': 1000,
    'save_epoch_freq':1,
    'no_html': True,
    'verbose': True,
    'suffix': ''
}

In [ ]:
def to_numpy_img(tensor):
    tensor = tensor.detach().cpu().clamp(0, 1)  # ensure valid range
    return tensor.permute(1, 2, 0).numpy()
tf=transforms.ToTensor()
topilimage = torchvision.transforms.ToPILImage()
dwi_sr_model = RealESRGAN(device, scale=2).model.to(device)
dwi_sr_model.load_state_dict(torch.load('../../model/ESRGAN/SWI/ckpt_45_checkpoint.pt',map_location=device))
ct_sr_model = RealESRGAN(device, scale=2).model.to(device)
ct_sr_model.load_state_dict(torch.load('../../model/ESRGAN/CT/ckpt_1000_checkpoint.pt',map_location=device))
tr_model = CycleGANModel()
from types import SimpleNamespace
opt = SimpleNamespace(**params)

tr_model.initialize(opt)
tr_model.setup(opt)
tr_model.device = device
tr_model.load_networks(413)
data_path='../../result/generator/Ischemic_DWI/'
save_path='../../result/generator_sr/'
nii_list=glob(data_path+'*.nii.gz')
nii_mask_list=[f.replace('/Ischemic_DWI','/Ischemic_mask') for f in nii_list]
step=0
for i in tqdm(range(len(nii_list))):
    nii_img=nib.load(nii_list[i])
    nii_img_data=(nii_img.get_fdata()+1.)/2.
    nii_img_mask=nib.load(nii_mask_list[i])
    nii_img_mask_data=nii_img_mask.get_fdata()
    step+=1
    folder_name=str(step).zfill(6)
    tensor_A=tf(nii_img.get_fdata()).float().to(device)
    a=tensor_A.permute(1, 2, 0).unsqueeze(0).unsqueeze(0)
    with torch.no_grad():
        fake_CT = tr_model.netG_A(a)# Clamp to 0~1 range
        fake_CT=(fake_CT+1.)/2
        fake_CT=fake_CT.squeeze()
    dwi_slide=[]
    ct_slide=[]
    mask_slide=[]
    for j in range(len(nii_img_data)):  # len(nii_img_data)
        image=tf(topilimage(nii_img_data[j]).convert('RGB')).float().to(device)
        image=image.unsqueeze(0)
        ct_image=tf(topilimage(fake_CT[j].squeeze().cpu()).convert('RGB')).float().to(device).unsqueeze(0)
        sr_mask=cv2.resize(nii_img_mask_data[j],(256,256))
        sr_mask=np.where(sr_mask==2,1,0)*255
    
        mask_slide.append(sr_mask)
        with torch.no_grad():
            sr = dwi_sr_model(image)
            sr = sr.squeeze(0).cpu().numpy()
            sr_ct=ct_sr_model(ct_image)
            sr_ct = sr_ct.squeeze(0).cpu().numpy()
            dwi_slide.append(sr[0])
            ct_slide.append(sr_ct[0])
        
    dwi_slide=np.array(dwi_slide)
    # min-max 정규화
    dwi_slide = (dwi_slide - dwi_slide.min()) / (dwi_slide.max() - dwi_slide.min())
    dwi_slide=dwi_slide*255
    dwi_slide = dwi_slide.astype(np.uint8)
    ct_slide=np.array(ct_slide)

    ct_slide = (ct_slide - ct_slide.min()) / (ct_slide.max() - ct_slide.min())
    ct_slide=ct_slide*255
    ct_slide = ct_slide.astype(np.uint8)
    mask_slide=np.array(mask_slide).astype(np.uint8)
    create_dir(save_path+'Ischemic_DWI/'+folder_name)
    create_dir(save_path+'Ischemic_CT/'+folder_name)
    create_dir(save_path+'Ischemic_mask/'+folder_name)
    for j in range(len(nii_img_data)):
        Image.fromarray(dwi_slide[j]).save(save_path+'Ischemic_DWI/'+folder_name+'/'+str(j).zfill(6)+'.png')
        Image.fromarray(ct_slide[j]).save(save_path+'Ischemic_CT/'+folder_name+'/'+str(j).zfill(6)+'.png')
        Image.fromarray(mask_slide[j]).save(save_path+'Ischemic_mask/'+folder_name+'/'+str(j).zfill(6)+'.png')

<font size="6">Normal</font>

In [ ]:
def to_numpy_img(tensor):
    tensor = tensor.detach().cpu().clamp(0, 1)  # ensure valid range
    return tensor.permute(1, 2, 0).numpy()
tf=transforms.ToTensor()
topilimage = torchvision.transforms.ToPILImage()
dwi_sr_model = RealESRGAN(device, scale=2).model.to(device)
dwi_sr_model.load_state_dict(torch.load('../../model/ESRGAN/SWI/ckpt_45_checkpoint.pt',map_location=device))
ct_sr_model = RealESRGAN(device, scale=2).model.to(device)
ct_sr_model.load_state_dict(torch.load('../../model/ESRGAN/CT/ckpt_1000_checkpoint.pt',map_location=device))

from types import SimpleNamespace
opt = SimpleNamespace(**params)
# tr_model = CycleGANModel()
# tr_model.initialize(opt)
# tr_model.setup(opt)
# tr_model.device = device
# tr_model .load_networks(153)
data_path='../../result/generator/Normal_DWI/'
save_path='../../result/generator_sr/'
nii_list=glob(data_path+'*.nii.gz')
nii_mask_list=[f.replace('/Normal_DWI','/Normal_mask') for f in nii_list]
step=0
for i in tqdm(range(len(nii_list))):
    nii_img=nib.load(nii_list[i])
    nii_img_data=(nii_img.get_fdata()+1.)/2.
    nii_img_mask=nib.load(nii_mask_list[i])
    nii_img_mask_data=nii_img_mask.get_fdata()
    step+=1
    folder_name=str(step).zfill(6)
    tensor_A=tf(nii_img.get_fdata()).float().to(device)
    a=tensor_A .permute(1, 0, 2).unsqueeze(0).unsqueeze(0)
    with torch.no_grad():
        fake_CT = tr_model.netG_A(a)# Clamp to 0~1 range
        fake_CT=torch.where(fake_CT<-1,-1,fake_CT)
        fake_CT=torch.where(fake_CT>1,1.,fake_CT)
        fake_CT=(fake_CT+1.)/2
        fake_CT=fake_CT.squeeze()
    dwi_slide=[]
    ct_slide=[]
    mask_slide=[]
    for j in range(len(nii_img_data)):  # len(nii_img_data)
        image=tf(topilimage(nii_img_data[j]).convert('RGB')).float().to(device)
        image=image.unsqueeze(0)
        ct_image=tf(topilimage(fake_CT[j].squeeze().cpu()).convert('RGB')).float().to(device).unsqueeze(0)
        sr_mask=cv2.resize(nii_img_mask_data[j],(256,256))
        sr_mask=np.where(sr_mask==2,1,0)*255
    
        mask_slide.append(sr_mask)
        with torch.no_grad():
            sr = dwi_sr_model(image)
            sr = sr.squeeze(0).cpu().numpy()
            sr_ct=ct_sr_model(ct_image)
            sr_ct = sr_ct.squeeze(0).cpu().numpy()
            dwi_slide.append(sr[0])
            ct_slide.append(sr_ct[0])
        
    dwi_slide=np.array(dwi_slide)
    # min-max 정규화
    dwi_slide = (dwi_slide - dwi_slide.min()) / (dwi_slide.max() - dwi_slide.min())
    dwi_slide=dwi_slide*255
    dwi_slide = dwi_slide.astype(np.uint8)
    ct_slide=np.array(ct_slide)

    ct_slide = (ct_slide - ct_slide.min()) / (ct_slide.max() - ct_slide.min())
    ct_slide=ct_slide*255
    ct_slide = ct_slide.astype(np.uint8)
    mask_slide=np.array(mask_slide).astype(np.uint8)
    create_dir(save_path+'Normal_DWI/'+folder_name)
    create_dir(save_path+'Normal_CT/'+folder_name)
    create_dir(save_path+'Normal_mask/'+folder_name)
    for j in range(len(nii_img_data)):
        Image.fromarray(dwi_slide[j]).save(save_path+'Normal_DWI/'+folder_name+'/'+str(j).zfill(6)+'.png')
        Image.fromarray(ct_slide[j]).save(save_path+'Normal_CT/'+folder_name+'/'+str(j).zfill(6)+'.png')
        Image.fromarray(mask_slide[j]).save(save_path+'Normal_mask/'+folder_name+'/'+str(j).zfill(6)+'.png')

In [ ]:
from torchinfo import summary
summary(tr_model.netG_A, input_size=(1, 1, 64, 128, 128), device=opt.device)